In [49]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [50]:
pdf_path = 'C:/Users/Jana/OneDrive/Dokumente/Desktop/Uni/Master/Thesis/PaperDatenextraktion/IARJSET7.pdf'
#pdf_path = open(str(filePath)+'Data Mining Tools.pdf', 'rb')

doc = fitz.open(pdf_path)
page = doc[2]
text = page.get_text("text")
print(doc.metadata)

{'format': 'PDF 1.5', 'title': 'Review Paper', 'author': 'Jayesh Minase', 'subject': 'Research Methodology', 'keywords': 'STP', 'creator': 'Microsoft® Office Word 2007', 'producer': 'Microsoft® Office Word 2007', 'creationDate': 'D:20180207133638', 'modDate': 'D:20180207133638', 'trapped': '', 'encryption': None}


In [44]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [45]:
get_entities(text)

['2019 Omkar user', 'downloaded  file']

In [39]:
entity_pairs = []

for i in tqdm(text["sentence"]):
  entity_pairs.append(get_entities(i))

TypeError: string indices must be integers